In [1]:
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np

# Load pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', force_download=True)
model = AutoModel.from_pretrained('distilbert-base-uncased', force_download=True)

E:\Python\raptor\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

E:\Python\raptor\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mateu\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

E:\Python\raptor\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [2]:
documents = [
    "TechGadget offers a refund within 30 days of purchase. The product must be in its original condition and "
    "packaging. Customers should provide proof of purchase to be eligible for a refund.",
    "SuperShop allows returns within 45 days of purchase, given the item is unused and in its original packaging."
    " Customers need to present a receipt or proof of purchase.",
    "QuickMart provides a refund within 15 days of purchase. Items should be unopened and accompanied by the"
    " original receipt."
]

In [3]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

doc_embeddings = np.vstack([embed_text(doc) for doc in documents])
doc_embeddings

array([[ 0.0456419 , -0.04133281,  0.29651454, ..., -0.07207134,
        -0.1411993 ,  0.14906308],
       [-0.10371029,  0.01745053,  0.2695513 , ..., -0.01926587,
        -0.1909957 ,  0.07620174],
       [-0.04579757, -0.08092573,  0.27003127, ...,  0.05650893,
        -0.22501247,  0.09502549]], dtype=float32)

In [4]:
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000028A391E7270> >

In [5]:
query = "What is the maximum amount of days for the refund policy of TechGadget?"
query_embedding = embed_text(query)
query_embedding

array([[ 1.14868417e-01, -2.02150643e-01,  2.88285285e-01,
         1.51861712e-01,  3.27129275e-01, -1.39032468e-01,
        -5.51950857e-02,  3.41996789e-01, -2.28577316e-01,
         1.08522065e-01,  3.16587575e-02, -9.66859236e-02,
        -6.39786944e-02,  3.09031516e-01, -1.64446801e-01,
         4.04367782e-02,  2.81969309e-02,  8.88244957e-02,
        -5.55643924e-02, -4.22496423e-02, -1.04028463e-01,
         2.23281756e-02,  3.79181169e-02,  3.09933245e-01,
         3.37026656e-01,  1.23849541e-01,  1.68965876e-01,
         2.35772282e-02, -2.03491613e-01, -8.98242295e-02,
         2.63925463e-01,  2.06674993e-01, -3.13754886e-01,
        -1.00394143e-02,  8.73428360e-02,  1.03583075e-01,
        -6.22465126e-02, -1.33356471e-02, -1.80988669e-01,
        -7.07881479e-03, -5.05747259e-01, -1.19636185e-01,
        -1.24004938e-01,  7.89727420e-02, -3.01284283e-01,
        -1.59267768e-01,  3.93764645e-01, -2.73394495e-01,
         2.34463692e-01, -3.41973543e-01, -3.51584077e-0

In [ ]:
k = 1
distances, indices = index.search(query_embedding, k)

In [ ]:
distances